# Importar librerias

In [2]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
## recuperar datos
corpus_df = pd.read_csv('./data/movieDescriptionDataSet.tsv', sep="\t", encoding ='latin1')

In [16]:
corpus_df.head()

,Title,Description,Rating,Link,Date,Unnamed: 5
0,Venom,Eddie Brock is a reporterinvestigating people...,65.0,/movie/335983,"October 5, 2018",NaN
1,Fantastic Beasts: The Crimes of Grindelwald,Gellert Grindelwald has escaped imprisonment a...,70.0,/movie/338952,"November 16, 2018",NaN
2,Robin Hood,A war-hardened Crusader and his Moorish comman...,62.0,/movie/375588,"November 21, 2018",NaN
3,The Seven Deadly Sins: Prisoners of the Sky,"Traveling in search of the rare ingredient, s...",57.0,/movie/507569,"August 18, 2018",NaN
4,Bohemian Rhapsody,"Singer Freddie Mercury, guitarist Brian May, d...",82.0,/movie/424694,"November 2, 2018",NaN


In [18]:
review = corpus_df['Description']

## Pre-Procesamiento

In [19]:
wpt = nltk.WordPunctTokenizer()
stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = wpt.tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

In [25]:
norm_corpus = normalize_corpus(review)
norm_corpus[0]

'eddie brock reporterinvestigating people want go unnoticed makes terrible discovery life foundation , begins transform \x91 venom \x92. foundation discovered creatures'

## Modelo de bolsa de palabras

In [21]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0., max_df=1.)
cv_matrix = cv.fit_transform(norm_corpus)
matrix_terms = np.array(cv.get_feature_names())

# Use the axis keyword to sum over rows
matrix_freq = np.asarray(cv_matrix.sum(axis=0)).ravel()
final_matrix = np.array([matrix_terms,matrix_freq])

#cv_matrix = cv_matrix.toarray()
#cv_matrix
final_matrix

array([['108', '11th', '16', ..., 'youth', 'zhao', 'zootopia'],
       ['1', '1', '2', ..., '1', '1', '1']], dtype='<U21')

In [22]:
vocab = cv.get_feature_names()
pd.DataFrame(final_matrix, columns=vocab)

,108,11th,16,16year,1847,1905,1926,1940s,1945,1955,...,yet,yeteradicating,yeti,york,young,younger,youngest,youth,zhao,zootopia
0,108,11th,16,16year,1847,1905,1926,1940s,1945,1955,...,yet,yeteradicating,yeti,york,young,younger,youngest,youth,zhao,zootopia
1,1,1,2,1,1,1,1,1,1,1,...,4,1,2,3,20,1,1,1,1,1


In [23]:
bv = CountVectorizer(ngram_range=(2,2))
bv_matrix = bv.fit_transform(norm_corpus)
bv_matrix = bv_matrix.toarray()
vocab = bv.get_feature_names()
pd.DataFrame(bv_matrix, columns=vocab)

,108 year,11th birthday,16 year,16year old,1847 ireland,1905 man,1926 newt,1940s double,1945 1955,1955 chronicle,...,young princess,young reporter,young thomas,young wizards,young yeti,younger selves,youngest son,youth current,zhao reprising,zootopia police
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(min_df=0., max_df=1., use_idf=True)
tv_matrix = tv.fit_transform(norm_corpus)
tv_matrix = tv_matrix.toarray()

vocab = tv.get_feature_names()
pd.DataFrame(np.round(tv_matrix, 2), columns=vocab)

,108,11th,16,16year,1847,1905,1926,1940s,1945,1955,...,yet,yeteradicating,yeti,york,young,younger,youngest,youth,zhao,zootopia
0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.2,0.0
7,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.43,0.0,0.13,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.00,0.0,0.0,...,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0
